In [ ]:
from importlib import reload
import xarray as xr
import dominosee.es as es

In [3]:
import dask.config
dask.config.set(scheduler='processes')

# Calculate ES on unstacked and stacked data

In [18]:
ds = xr.open_dataset("tests/data/era5.reanalysis.spi30d.0p25deg.china.1950-1979.nc").rename({"latitude": "lat", "longitude": "lon", "spi30d": "SPI1"})\
        .isel(lat=slice(0, 25), lon=slice(0, 20), time=slice(0, 300)).load()
ds_stack = ds.stack(location=["lat", "lon"])
ds

<xarray.Dataset> Size: 603kB
Dimensions:  (time: 300, lon: 20, lat: 25)
Coordinates:
  * time     (time) datetime64[ns] 2kB 1950-01-01 1950-01-02 ... 1950-10-27
  * lon      (lon) float64 160B 73.0 73.25 73.5 73.75 ... 77.0 77.25 77.5 77.75
  * lat      (lat) float64 200B 54.25 54.0 53.75 53.5 ... 49.0 48.75 48.5 48.25
Data variables:
    SPI1     (time, lat, lon) float32 600kB -0.531 -0.5337 ... -0.1255 -0.1956
Attributes:
    CDI:          Climate Data Interface version 2.4.4 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Thu Mar 06 20:30:32 2025: cdo -L selindexbox,293,542,144,34...
    CDO:          Climate Data Operators version 2.4.4 (https://mpimet.mpg.de...

In [24]:
reload(es)
event_ds = es.get_event_positions(ds.SPI1 < -1.5)
event_ds_stack = es.get_event_positions(ds_stack.SPI1 < -1.5)
event_ds

<xarray.Dataset> Size: 225kB
Dimensions:          (lon: 20, lat: 25, event: 110)
Coordinates:
  * lon              (lon) float64 160B 73.0 73.25 73.5 ... 77.25 77.5 77.75
  * lat              (lat) float64 200B 54.25 54.0 53.75 ... 48.75 48.5 48.25
  * event            (event) int64 880B 0 1 2 3 4 5 ... 104 105 106 107 108 109
Data variables:
    event_positions  (lat, lon, event) int32 220kB 37 38 39 40 ... -1 -1 -1 -1
    event_count      (lat, lon) int64 4kB 32 33 33 32 32 32 ... 45 51 58 22 20

In [25]:
event_diff_da = es.get_event_time_differences(event_ds.event_positions, event_ds.event_count)
event_diff_da_stack = es.get_event_time_differences(event_ds_stack.event_positions, event_ds_stack.event_count)
event_diff_da

<xarray.DataArray 'event_positions' (lat: 25, lon: 20, event: 110)> Size: 440kB
array([[[ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        ...,
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan]],

       [[ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        ...,
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [18., 32.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan]],

       [[ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        ...,
...
        ...,
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan]],

       [[ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        ...,
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan]],

       [[ 1.,  1., 95., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ...,  1.,  1., nan],
        ...,
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan]]], shape=(25, 20, 110))
Coordinates:
  * lon      (lon) float64 160B 73.0 73.25 73.5 73.75 ... 77.0 77.25 77.5 77.75
  * lat      (lat) float64 200B 54.25 54.0 53.75 53.5 ... 49.0 48.75 48.5 48.25
  * event    (event) int64 880B 0 1 2 3 4 5 6 7 ... 103 104 105 106 107 108 109
Attributes:
    long_name:    Event Time Differences
    units:        time steps
    description:  Time differences between consecutive events for each locati...

In [26]:
reload(es)
es_ds = es.get_event_sync_from_positions(event_ds.event_positions, event_ds.event_positions.copy(), 
                                 event_diff_da, event_diff_da.copy(),
                                 event_ds.event_count, event_ds.event_count.copy(),
                                 )
es_ds_stack = es.get_event_sync_from_positions(event_ds_stack.event_positions, event_ds_stack.event_positions.copy(), 
                                 event_diff_da_stack, event_diff_da_stack.copy(),
                                 event_ds_stack.event_count, event_ds_stack.event_count.copy(),
                                 )
es_ds_stack

<xarray.DataArray (locationA: 500, locationB: 500)> Size: 250kB
array([[30, 30, 30, ...,  0,  0,  0],
       [30, 31, 31, ...,  0,  0,  0],
       [30, 31, 31, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ..., 56, 21, 19],
       [ 0,  0,  0, ..., 20, 20, 13],
       [ 0,  0,  0, ..., 18, 12, 18]], shape=(500, 500), dtype=uint8)
Coordinates:
  * locationA  (locationA) object 4kB MultiIndex
  * latA       (locationA) float64 4kB 54.25 54.25 54.25 ... 48.25 48.25 48.25
  * lonA       (locationA) float64 4kB 73.0 73.25 73.5 ... 77.25 77.5 77.75
  * locationB  (locationB) object 4kB MultiIndex
  * latB       (locationB) float64 4kB 54.25 54.25 54.25 ... 48.25 48.25 48.25
  * lonB       (locationB) float64 4kB 73.0 73.25 73.5 ... 77.25 77.5 77.75
Attributes:
    long_name:    Event Synchronization
    units:        count
    description:  Number of synchronized events between locations A and B
    threshold:    inf

In [23]:
assert es_ds.stack(locationA=["latA", "lonA"], locationB=["latB", "lonB"]).equals(es_ds_stack)